<h1> Assignment #1 </h1>

You are an engineer in the logistic company. You have a robot which is capable of cargo transportation. Now you use it to carry the equipment over the cluttered warehouse. 

Your task is to train bot to find the shortest path to the required location (lower right corner) so that it does not crash into obstacles.  

---

Submit `{name}_{surname}.py` script with `find_path(path_to_infile, path_to_outfile)` function.

**You have to solve it using MDP.**

In [ ]:
from typing import List, Tuple
import numpy as np

In [ ]:
class GridWorld:        
    def __init__(self, n: int, m: int,
                 gamma: float = 0.9,
                 default_reward: float = -1.,
                 cargo: List[Tuple[int, int]] = [],
                 obstacles: List[Tuple[int, int]] = [],
                 terminate_states: List[Tuple[int, int]] = [],
                 ):
        self.n = n # number of rows
        self.m = m # number of columns
        self.board = [(i, j) for i in range(n) for j in range(m)]
        self.gamma = gamma
        self.default_reward = default_reward
        self.cargo = cargo # coordinates of all cargo's cells
        self.cargo_pivot = self.get_cargo_pivot()
        self.obstacles = obstacles # cooridnates of all obstacles' cells
        self.states = set(self.board).difference(
            set(self.obstacles)
            ) | set(self.cargo)
        self.svfs = {state: 0. for state in self.states} # state value funtctions
        self.terminate_states = set(terminate_states)
        self.policy = {cell: '.' for cell in self.board}


    def train_agent(self):
        eps = 10e-3 # epsilon to finish computation

        def svfs_diff(prev, next):
            res = []
            for key, _ in prev.items():
                res.append(abs(prev[key] - next[key]))
            return res

        t = 0
        while True:
            t += 1
            prev_svfs = self.svfs.copy()
            self.update_svfs() # execute term 0
            diff = svfs_diff(prev_svfs, self.svfs)
            if all(map(lambda x: x < eps, diff)):
                break
        
        print(f"Finished at a time step #{t}")
        print(self)
        self.show_svfs()
        self.show_policy()

        def get_path(policy, pivot, terminates):
            """
            Идёт от пивота по указанным policy,
            если два раза и более зашёл в одно и то же
            состояние, значит пути нет.
            """
            res = set()
            path = []
            actions = {
            "L": (0, -1),
            "R": (0, +1),
            "U": (-1, 0),
            "D": (+1, 0)
            }
            while True:
                if policy[pivot] == '.' and pivot not in terminates:
                    return "No path"
                if pivot in res:
                    return "No path"
                if pivot in terminates:
                    break
                res.add(pivot)

                x, y = pivot
                action = policy[pivot]
                x_d, y_d = actions[action]
                path.append(action)

                pivot = x + x_d, y + y_d
            
            return path
        
        self.path = get_path(self.policy, 
                             self.cargo_pivot,
                             self.terminate_states)


    def update_svfs(self):
        """
        Update state-value functions for current time step t.
        """
        new_svfs = self.svfs.copy()

        for state in self.states:
            if state in self.terminate_states:
                continue
            action_name, svf = self.compute_svf(state)
            new_svfs[state] = svf
            self.policy[state] = action_name

        self.svfs = new_svfs


    def compute_svf(self, state):
        """
        Compute state-value function for current time step t.
        """
        qvfs = []
        states = self.possible_actions(state) # new possible states
        for action_name, to_state in states:
            svf = self.default_reward + self.gamma * self.svfs[to_state]
            qvfs.append((action_name, svf))

        if not qvfs:
            return self.policy[state], self.svfs[state]
        return max(qvfs, key=lambda el: el[1])


    def possible_actions(self, from_state) -> List[Tuple[int, int]]:
        actions = {
            "L": (0, -1),
            "R": (0, +1),
            "U": (-1, 0),
            "D": (+1, 0)
            }
        x, y = from_state
        
        ret = []
        for name, action in actions.items():
            x_d, y_d = action
            to_state = x + x_d, y + y_d
            if self.is_action_possible(to_state):
                ret.append((name, to_state))
        
        return ret


    def is_action_possible(self, to_state) -> bool:
        """
        Надо проверить, не ломается ли фигура, при перемещении
        в данную ячейку, и не выходит ли за границы мира.
        """
        x, y = self.cargo_pivot
        x_prime, y_prime = to_state
        x_d, y_d = x_prime - x, y_prime - y # считаем насколько переместились
        cargo = [(x + x_d, y + y_d) for x, y in self.cargo] # считаем для каждой
                                                            # ячейки груза её 
                                                            # новые координаты
        
        in_boundaries = all(map(
            lambda el: 0 <= el[0] < self.n and 0 <= el[1] < self.m, cargo)
            )
        if not in_boundaries:
            return False
        
        for cell in cargo:
            # if cell in self.cargo and cell != self.cargo_pivot:
                # return False
            if cell in self.obstacles:
                return False
        
        return True


    def get_cargo_pivot(self):
        pivot, *_ = sorted(
            self.cargo, key=lambda el: (el[1], el[0]), reverse=True
            ) # берём максимальную по y и максимальную по x ячейку за опорную
              # (как если бы мы за краешек фигуры перенесли её в новую точку)
        return pivot


    def show_svfs(self):
        grid = np.zeros((self.n, self.m))
        for state, value in self.svfs.items():
            x, y = state
            grid[x, y] = value
        
        print(grid)
    

    def show_policy(self):
        grid = np.empty([self.n, self.m], dtype=str)
        for state, action in self.policy.items():
            x, y = state
            grid[x, y] = action
        
        print(grid)


    def __str__(self):
        grid = np.zeros((self.n, self.m))
        for x, y in self.cargo:
            grid[x, y] = 2
        for x, y in self.obstacles:
            grid[x, y] = 1
        return str(grid)
    

    def __repr__(self):
        return str(self)

In [ ]:
cargo = [(0, 0)]#, (1, 0), (2, 0)]
obstacles = [(0, 1), (1, 1), (3, 1), (4, 1), (0, 3), (1, 3), (3, 3), (4, 3), (4, 4)]
terminate_states = [(4, 4)]

world = GridWorld(5, 5, 
                  gamma=0.9,
                  default_reward=-1., 
                  cargo=cargo,
                  obstacles=obstacles,
                  terminate_states=terminate_states)
world.svfs[(4, 4)] = 10
world

[[2. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 1.]]

In [ ]:
T = 100
for t in range(T):
    # print(f"\nTime step #{t}")
    # world.show_svfs()
    # world.show_policy()
    world.update_svfs()
world.show_svfs()
world.show_policy()

[[-9.99973439  0.         -9.99973439  0.         -9.99973439]
 [-9.99973439  0.         -9.99973439  0.         -9.99973439]
 [-9.99973439 -9.99973439 -9.99973439 -9.99973439 -9.99973439]
 [-9.99973439  0.         -9.99973439  0.         -9.99973439]
 [-9.99973439  0.         -9.99973439  0.         10.        ]]
[['D' '.' 'D' '.' 'D']
 ['U' '.' 'U' '.' 'U']
 ['R' 'L' 'L' 'L' 'L']
 ['U' '.' 'U' '.' 'U']
 ['U' '.' 'U' '.' '.']]


In [ ]:
def find_path(infile, outfile):
    grid = []
    with open(f"{infile}.txt", 'r') as f:
        for line in f:
            grid.append(line.split())
    
    cargo = []
    obstacles = []
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == '1':
                obstacles.append((i, j))
            elif grid[i][j] == '2':
                cargo.append((i, j))
    
    n, m = len(grid), len(grid[0])
    terminate_states = [(n - 1, m - 1)]
    world = GridWorld(n, m, 
                  gamma=0.9,
                  default_reward=-1., 
                  cargo=cargo,
                  obstacles=obstacles,
                  terminate_states=terminate_states)
    
    world.svfs[terminate_states[-1]] = 10
    world.train_agent()
    print(world.path)
    

In [ ]:
find_path("test", "")

Finished at a time step #9
[[2. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]]
[[-1.3906558  0.         0.62882    0.         3.122    ]
 [-0.434062   0.         1.8098     0.         4.58     ]
 [ 0.62882    1.8098     3.122      4.58       6.2      ]
 [-0.434062   0.         1.8098     0.         8.       ]
 [-1.3906558  0.         0.62882    0.        10.       ]]
[['D' '.' 'D' '.' 'D']
 ['D' '.' 'D' '.' 'D']
 ['R' 'R' 'R' 'R' 'D']
 ['U' '.' 'U' '.' 'D']
 ['U' '.' 'U' '.' '.']]
['D', 'D', 'R', 'R', 'R', 'R', 'D', 'D']


### Input:

`{infile}.txt` file with the field description. Elements of the field are separated by space. For example: 
```
2 1 0 1 0
0 1 0 1 0
0 0 0 0 0
0 1 0 1 0
0 1 0 1 0
```

* `0` - blank space, we may move objects here
* `1` - obstackles, object can not be over that position
* `2` - actual object shape, does not change, moved as a solid object

As our goal is to move object to the most lower-right position, sequence of our steps will be the following:  
`D D R R R R D D`

Meaning:  
`D` - (move) Down;  
`U` - (move) Up;  
`R` - (move) Right;  
`L` - (move) Left.

### Output:

Sequence that leads to the right lower corner in the least number of steps. Written in `{outfile}.txt`. If there is no path, write `No path` in the file.

### Examples

#### First case

`input.txt`:

```
0 1 0 1 0
0 1 0 1 0
0 0 0 0 0
0 0 0 0 0
0 1 2 1 0
0 1 2 0 0
```

`out.txt`:

```
U U R R D D
```

#### Second case

`input.txt`:

```
2 2 1 0 1 0
0 2 1 0 1 0
0 2 0 0 0 0
0 0 0 0 0 0
0 0 1 0 1 0
0 0 1 0 0 0
```

`out.txt`:

```
No path
```

#### Third case

`input.txt`:

```
2 2 1 0 1 0
0 2 1 0 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 1 0 1 0
0 0 1 0 0 0
```

`out.txt`:

```
No path
```

#### Fourth case

`input.txt`:

```
2 2 1 0 1 0
0 2 0 0 0 0
0 2 0 0 0 0
0 0 0 0 0 0
0 0 0 0 1 0
0 0 1 0 0 0
```

`out.txt`:

```
D D R R U R R D D
```